> Please go through Giba's post and kernel  to underrstand what this leak is all about
> https://www.kaggle.com/titericz/the-property-by-giba (kernel)
> https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)
> 
> Also, go through this Jiazhen's kernel which finds more columns to exploit leak
> https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
> 
> I just exploit data property in brute force way and then fill in remaining by row non zero means! This should bring everyone on level-playing field.
> 
> **Let the competition begin! :D**

### Just some small modifications from [original baseline](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)~
- The leak rows are calculated separately on train/test set
- Calculated the leaky values, correctness, for each lag
- Hope this can help to do some *lag_selection*

### Update leak process codes to Dmitry Frumkin's *fast* [version](https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2)
- The result of Dmitry's original function and result of Hasan's function seem slightly different
- Modified to make the output consistent with Hasan's function (Seems better score)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [4]:
test["target"] = train["target"].mean()

# Determine Extra Features to use

In [36]:
rows = [1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245, 1383, 2071, 3492, 378, 2971, 2366, 4414, 2790, 3979, 193, 1189, 3516, 810, 4443, 3697, 235, 1382, 4384, 3418, 4396, 921, 3176, 650]

def element_wise_comparison(i1, i2, thresh=3):
    i1 = np.array(i1)
    i2 = np.array(i2)
    dif = i1 - i2
    count = np.count_nonzero(dif)
    if count <= thresh:
        return True
    else:
        return False

def solve_ties_in_nb_of_fillers(df, counts, stop_thresh=2):
    args = np.argsort(counts)[::-1]
    stop = np.squeeze(np.where(counts[args] == len(df.index)-stop_thresh))
    try:
        stop = int(stop)
    except:
          return [None]
    u, c = np.unique(counts[args[stop:]], return_counts=True)
    conflicts = u[np.where(c > 1)]
    for c in conflicts:
        idx = np.squeeze(np.where(counts == c))
        ref_id = np.squeeze(np.where(counts == c+1))
        ref = df.iloc[:-1, int(ref_id)]
        dif1 = np.count_nonzero(df.iloc[1:, idx[0]].values - ref.values)
        dif2 = np.count_nonzero(df.iloc[1:, idx[1]].values - ref.values)
        loc0  = np.where(args == idx[0])
        loc1  = np.where(args == idx[1])
        tmp0 = args[loc0]
        tmp1 = args[loc1]

        if dif2 < dif1:
            args[np.minimum(loc0, loc1)] = tmp1
            args[np.maximum(loc0, loc1)] = tmp0
        else:
            args[np.minimum(loc0, loc1)] = tmp0
            args[np.maximum(loc0, loc1)] = tmp1           

    return args[stop:]
    
        
def find_filler_value(df, idx):
    u, c = np.unique(df.iloc[0,:].values, return_counts=True)
    arg = np.argsort(c)
    return u[arg][-idx], c[arg][-idx]

def find_col_order_by_filler(df, filler, stop_thresh=2, nb=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = solve_ties_in_nb_of_fillers(df, counts, stop_thresh)
    if not all(args):
        return [None]
    return df.iloc[:, args]

def find_leading_col(raw_df, sorted_df, thresh=3):
    curr_lead = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[0, 0][1:]
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for t in range(thresh+1):
        for c in cols_to_match.index:
                if element_wise_comparison(cols_to_match.loc[c, 'key'][:-1], curr_lead, thresh=t):
                    print('Leading col found: {}'.format(c))
                    return pd.concat([raw_df[c], sorted_df], axis=1)
        
    print('No lead found')
    return sorted_df
    
def find_n_cols_groups(df, n_groups=40, stop_thresh=2, col_count_tresh=30):
    col_groups = []
    for i in range(n_groups):
        filler, count = find_filler_value(df, i)
        print(filler, count)
        if filler == 0 or count <10:
            continue
        df_view = find_col_order_by_filler(df, filler)
        if not all(df_view):
            extra_cols = find_col_order_by_filler_for_ambiguous_cols(df, filler)
            col_groups.extend(extra_cols)
        else:
            for _ in range(stop_thresh + 1):
                df_view = find_leading_col(df, df_view)
            col_groups.append(list(df_view.columns))
            
    # Remove cols groups with less cols than tresh
    col_groups = [x for x in col_groups if len(x) >= col_count_tresh]
    return col_groups

def find_bottom_col(raw_df, sorted_df, thresh=4):
    curr_bot = sorted_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'}).iloc[-1, 0][:-1]
#     print(curr_bot)
    cols_to_match = raw_df.apply(tuple, axis=0).to_frame().rename(columns={0: 'key'})
    for t in range(thresh):
        for c in cols_to_match.index:
    #         print(cols_to_match.loc[c, 'key'][1:])
            if element_wise_comparison(cols_to_match.loc[c, 'key'][1:], curr_bot, thresh=t):
                print('Bottom col found: {}'.format(c))
                return pd.concat([sorted_df, raw_df.pop(c)], axis=1), True
    
    print('No bottom found')
    return sorted_df, False

def remove_unsutable_candidates(candidates_pool, ref,  thresh=1):
    unique = ref.iloc[:, 0].unique()
    for c in candidates_pool.columns:
        tmp =  0
        for u in unique:
            if u not in candidates_pool[c].values:
                tmp +=1 
            if tmp > thresh:
                candidates_pool = candidates_pool.drop(c, axis=1)
                break
    return candidates_pool
    
def find_col_order_by_filler_for_ambiguous_cols(df, filler, n_cols=40):
    counts = []
    for c in df.columns:
        v = df[c].values - filler
        counts.append(np.count_nonzero(v))
    counts = np.array(counts)
    args = np.argsort(counts)[::-1]
    stop = np.squeeze(np.where(counts[args] == len(df.index)-2))
    extra_cols = []
#     return df.iloc[:, args[min(stop):]]
    for j in range(len(stop)):
        sorted_cols = df.iloc[:, args[stop[j]]].to_frame()
        candidates_pool = df.iloc[:, args[min(stop):]].copy()
#         candidates_pool = remove_unsutable_candidates(candidates_pool, sorted_cols, 2)

        for _ in range(n_cols-3):
            sorted_cols, state = find_bottom_col(candidates_pool, sorted_cols)
            if not np.count_nonzero(sorted_cols.iloc[:, -1].values - filler) or not state:
                break

        for i in range(2):
            sorted_cols = find_leading_col(df, sorted_cols, thresh=3)
        extra_cols.append(list(sorted_cols.columns))
    return extra_cols

# Analysis

In [21]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
tmp = train.loc[rows, :].drop('ID', axis=1)
find_col_order_by_filler_for_ambiguous_cols(tmp, 1032000)[1]
# extra_feats = [list(x.columns) for x in find_col_order_by_filler_for_ambiguous_cols(tmp, 0)]
# [len(x) for x in extra_feats]

Bottom col found: 2d7f256c6
Bottom col found: e98dd91da
Bottom col found: bce33962d
Bottom col found: cdaf3e576
Bottom col found: 78b2cf944
Bottom col found: b3dbb1222
Bottom col found: a2d707f4e
Bottom col found: a5565ec7d
Bottom col found: 26b423c42
Bottom col found: 2731a96a2
Bottom col found: c24634d56
Bottom col found: dc61677e5
Bottom col found: 62c319fd1
Bottom col found: fbe583de5
Bottom col found: ab7534e84
Bottom col found: 92eb376f1
Bottom col found: 7def2d7d3
Bottom col found: 657d07d19
Bottom col found: df08959aa
Bottom col found: 48130b04b
Bottom col found: e0cad27c5
Bottom col found: 0b139c060
Bottom col found: 99df18cb2
Bottom col found: 32c6f9aa6
Bottom col found: 7ff7d351b
Bottom col found: e49fea26e
Bottom col found: 403e7e48d
Bottom col found: 2c5809e51
Bottom col found: fe1160758
Bottom col found: d83991b80
Bottom col found: ce999e374
Bottom col found: 29e58696d
Bottom col found: 212226b7b
Bottom col found: 950b9cf2d
Bottom col found: adc70e02f
Bottom col found: 77

,d48c44c49,e926d89d3,4c0845957,892f6f03e,61b88b77a,207021f50,bb12b7d99,e17a13988,bfe5872ea,788e6045c,8291f21d8,36ba6bac3,66e323ce9,a05f1bb14,b379107b3,084031585,571380d69,640a36efb,5a53ebb69,33288f3b2,32138f717,dd4649092,8f993c285,7b58b1340,a3fb71dbd,773c101b5,7f6e612c7,122df5341,e49fea26e,403e7e48d,2c5809e51,fe1160758,d83991b80,ce999e374,29e58696d,212226b7b,950b9cf2d,adc70e02f,7792cebe0,22f0deffc
1757,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3809,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
511,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3798,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
625,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3303,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
4095,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
1283,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
4209,0,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
1696,0,0,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000


In [6]:
tmp = train.loc[rows, :].drop('ID', axis=1)
u, c = np.unique(tmp.iloc[0,:].values, return_counts=True)
arg = np.argsort(c)[::-1]
u[arg], c[arg]
# u[np.where(c == 36)]

(array([        0.  ,   1032000.  ,   1500000.  ,   1674000.  ,
          4000000.  ,   3000000.  ,   2992315.78,   2865694.28,
          2831000.  ,   1775333.34,  45138000.  ,   2160000.  ,
          2002166.66,   3160000.  ,   2000000.  ,   1964666.66,
          2400000.  ,   1612000.  ,   3255483.88,   1563411.76,
          1470000.  ,   1166666.66,   1033000.  ,    812666.66,
           803538.46,    726000.  ,    662000.  ,    400000.  ,
           210000.  ,     80000.  ,   3249000.  ,  70448000.  ,
          2202500.  ,   6568857.14,  40770000.  ,   3822750.  ,
         14650000.  ,  11028000.  ,   4434140.  ,   7822000.  ,
          7440000.  ,   4677000.  ,   5300000.  ,   5422000.  ,
          5600000.  ,   5921285.72,   6902000.  ,    790000.  ,
           200000.  ,   1830000.  ,  34714000.  ,    604000.  ,
           253333.34,    273000.  ,    182000.  ,    278000.  ,
           174000.  ,    346000.  ,    360000.  ,    144000.  ,
           380000.  ,    120000.  ,    1

In [ ]:
tmp = test.loc[rows, :].drop('ID', axis=1)
u, c = np.unique(tmp.iloc[0,:].values, return_counts=True)
arg = np.argsort(c)[::-1]
u[arg], c[arg]

# Calculate extra features

In [28]:
tmp = train.loc[rows, :].drop('ID', axis=1)
extra_feats = find_n_cols_groups(tmp, 85)
# extra_feats
len(extra_feats)

1015000.0 1
0.0 3002
1032000.0 78
Bottom col found: 2d7f256c6
Bottom col found: e98dd91da
Bottom col found: bce33962d
Bottom col found: cdaf3e576
Bottom col found: 78b2cf944
Bottom col found: b3dbb1222
Bottom col found: a2d707f4e
Bottom col found: a5565ec7d
Bottom col found: 26b423c42
Bottom col found: 2731a96a2
Bottom col found: c24634d56
Bottom col found: dc61677e5
Bottom col found: 62c319fd1
Bottom col found: fbe583de5
Bottom col found: ab7534e84
Bottom col found: 92eb376f1
Bottom col found: 7def2d7d3
Bottom col found: 657d07d19
Bottom col found: df08959aa
Bottom col found: 48130b04b
Bottom col found: e0cad27c5
Bottom col found: 0b139c060
Bottom col found: 99df18cb2
Bottom col found: 32c6f9aa6
Bottom col found: 7ff7d351b
Bottom col found: e49fea26e
Bottom col found: 403e7e48d
Bottom col found: 2c5809e51
Bottom col found: fe1160758
Bottom col found: d83991b80
Bottom col found: ce999e374
Bottom col found: 29e58696d
Bottom col found: 212226b7b
Bottom col found: 950b9cf2d
Bottom col fou

Bottom col found: 3e1100230
Bottom col found: 82f715995
Bottom col found: 59cafde1f
Bottom col found: 1d81e197a
Bottom col found: 3f8854df3
Bottom col found: 17b81a716
Bottom col found: 26cc05472
Bottom col found: 6786ea46d
Bottom col found: 1110cf9ea
Bottom col found: 621833d9b
Bottom col found: 5a798adc1
Bottom col found: c270cb02b
Bottom col found: 26ab20ff9
Bottom col found: fbaed5913
Bottom col found: 1d9b78b9c
Leading col found: bbe4423a3
Leading col found: 9a9fc1aba
3000000.0 39
Leading col found: d8e951a15
Leading col found: 3391c0af1
No lead found
2992315.78 39
Leading col found: 9126049d8
Leading col found: a3e023f65
No lead found
2865694.28 39
Leading col found: ca7ea80a3
Leading col found: ccc7609f4
No lead found
2831000.0 39
Leading col found: 2b8851e90
Leading col found: 48b839509
No lead found
1775333.34 39
Leading col found: 63c094ba4
Leading col found: 87ffda550
No lead found
45138000.0 39
Leading col found: 93efdb50f
Leading col found: 920a04ee2
No lead found
2160000.

Bottom col found: 20ef8d615
No bottom found
Leading col found: 2b0fc604a
Leading col found: eb5a2cc20
1964666.66 39
Leading col found: 9df4daa99
Leading col found: ced6a7e91
No lead found
2400000.0 39
Leading col found: 34d3715d5
Leading col found: d0d340214
No lead found
1612000.0 39
Leading col found: c0004231c
Leading col found: 6cf7644e0
Leading col found: 48df886f9
3255483.88 39
Leading col found: 64e483341
Leading col found: 1d9078f84
No lead found
1563411.76 39
Leading col found: 58e2e02e6
Leading col found: f190486d6
No lead found
1470000.0 39
Leading col found: fa11da6df
Leading col found: f3cf9341c
No lead found
1166666.66 39
Leading col found: 8e4d0fe45
Leading col found: c928b4b74
No lead found
1033000.0 39
Leading col found: 0e348d340
Leading col found: 51c141e64
No lead found
812666.66 39
Leading col found: 8c5025c23
Leading col found: 1ad24da13
No lead found
803538.46 39
Leading col found: 75b663d7d
Leading col found: 4302b67ec
No lead found
726000.0 39
Leading col found

99

In [32]:

cols_2 = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11','73687e512',
                'aa164b93b','342e7eb03','cd24eae8a','8f3740670','2b2a10857','a00adf70e',
                '3a48a2cd2','a396ceeb9','9280f3d04','fec5eaf1a','5b943716b','22ed6dba3',
                '5547d6e11','e222309b0','5d3b81ef8','1184df5c2','2288333b4','f39074b55',
                'a8b721722','13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]
# x = extra_feats

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
for i, c in enumerate(extra_feats):
    if not i:
        x = pd.concat([train.loc[rows, ['target']+cols], train.loc[rows, c]], axis=1)
    else:
        x = pd.concat([x, train.loc[rows, c]], axis=1)
# display(train.loc[rows, extra_feats[25]])
display(x)

# Find leaks

In [29]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, extra_feats=[[]], lag=0):
    f1 = cols[:-lag-2]
    f2 = cols[lag+2:]
    for ef in extra_feats:
        if len(ef) <= 2:
            continue
        f1 += ef[:-lag-2]
        f2 += ef[lag+2:]
    
    
    d1 = df[f1].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[f2].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [30]:
def compiled_leak_result(extra_cols, max_nlags=38, verbose=True):    
    train_leak = train
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        if verbose:
            print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, extra_cols, i)
        
        leaky_cols.append(c)
#         train_leak = train.join(train_leak[leaky_cols+["compiled_leak", "nonzero_mean", "ID"]], on='ID', how='left')
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        if verbose:
            print("Leak values found in train", leaky_value_counts[-1])
            print(
                "% of correct leaks values in train ", 
                leaky_value_corrects[-1])
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = train_leak["nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        if verbose:
            print(
                'Score (filled with nonzero mean)', 
                scores[-1])
        gc.collect()
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

# Find helpful cols

In [42]:
good_cols = []
bad_cols = []
_, result = compiled_leak_result([[]], max_nlags=38, verbose=False)
ref = result['score']
for i in range(len(extra_feats)):
    extra_cols = [extra_feats[i]]
    _, result = compiled_leak_result(extra_cols, max_nlags=3, verbose=False)
    if min(result['score']) < ref:
        print('Good col found, leader is ', extra_feats[i][0])
        good_cols.append(extra_feats[i])
    else:
        bad_cols.append(extra_feats[i])
print(len(good_cols), ' good columns found!')

Good col found, leader is  6469d38ef
Good col found, leader is  a3e023f65
Good col found, leader is  ccc7609f4
Good col found, leader is  48b839509
Good col found, leader is  87ffda550
Good col found, leader is  920a04ee2
Good col found, leader is  9d4428628
Good col found, leader is  5030aed26
Good col found, leader is  81de0d45e
Good col found, leader is  a5f8c7929
Good col found, leader is  ced6a7e91
Good col found, leader is  d0d340214
Good col found, leader is  1d9078f84
Good col found, leader is  f3cf9341c
Good col found, leader is  c928b4b74
Good col found, leader is  51c141e64
Good col found, leader is  1ad24da13
Good col found, leader is  4302b67ec
Good col found, leader is  63be1f619
Good col found, leader is  7ba58c14d
Good col found, leader is  2135fa05a
Good col found, leader is  b6daeae32
Good col found, leader is  fec5644cf
Good col found, leader is  0f8d7b98e
Good col found, leader is  a1cd7b681
Good col found, leader is  5b465f819
Good col found, leader is  df838756c
G

In [91]:
_, result = single_leak_result([[]], lag=10, verbose=True)

Processing lag 10
Leak values found in train 1089
% of correct leaks values in train  0.998163452709
Score (filled with nonzero mean) 1.51383333916


In [95]:
# Apply fix to bad cols
def fix_bad_cols(df, bad_cols, verbose=True):
    _, result = compiled_leak_result([[]], max_nlags=38, verbose=verbose)
    ref = result['score']
    fixed_cols = []
    for i in range(len(bad_cols)):
        extra_cols = bad_cols[i]
        for j in range(len(extra_cols)-2):
            _, result = compiled_leak_result([extra_cols[:j+2]], max_nlags=j, verbose=verbose)
            if result['score'][j] > ref[j]:
                print('Bad cause found ', extra_cols[j+2])
                pool = extra_cols[j+1:]
                pool.append(pool.pop(0))
                for k in range(len(pool)):
                    _, result = compiled_leak_result([extra_cols[:j+1]+[pool[0]]], max_nlags=j, verbose=verbose)
                    if result['score'][j] < ref[j]:
                        print('Found new order ', pool[0])

In [96]:
x = fix_bad_cols(tmp, bad_cols)

Processing lag 0
Leak values found in train 1351
% of correct leaks values in train  0.9955588453
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 1947
% of correct leaks values in train  0.996404725218
Score (filled with nonzero mean) 1.29220481295
Processing lag 2
Leak values found in train 2340
% of correct leaks values in train  0.99358974359
Score (filled with nonzero mean) 1.17328290468
Processing lag 3
Leak values found in train 2586
% of correct leaks values in train  0.993039443155
Score (filled with nonzero mean) 1.08432637367
Processing lag 4
Leak values found in train 2754
% of correct leaks values in train  0.993464052288
Score (filled with nonzero mean) 1.032787044
Processing lag 5
Leak values found in train 2899
% of correct leaks values in train  0.993101069334
Score (filled with nonzero mean) 0.99403242995
Processing lag 6
Leak values found in train 3014
% of correct leaks values in train  0.992368944924
Score (filled with nonz

IndexError: list index out of range

In [65]:
# Visualize bad_cols to understand what is happenning
# len(bad_cols)
tmp.loc[:, bad_cols[0]]

,d48c44c49,e926d89d3,4c0845957,892f6f03e,61b88b77a,207021f50,bb12b7d99,e17a13988,bfe5872ea,788e6045c,8291f21d8,36ba6bac3,66e323ce9,a05f1bb14,b379107b3,084031585,571380d69,640a36efb,5a53ebb69,33288f3b2,32138f717,dd4649092,8f993c285,7b58b1340,a3fb71dbd,773c101b5,7f6e612c7,122df5341,e49fea26e,403e7e48d,2c5809e51,fe1160758,d83991b80,ce999e374,29e58696d,212226b7b,950b9cf2d,adc70e02f,7792cebe0,22f0deffc
1757,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3809,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
511,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3798,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
625,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
3303,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
4095,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
1283,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
4209,0,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000
1696,0,0,0,0,0,252000,0,0,0,0,0,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000,1032000


In [41]:
train_leak, result = compiled_leak_result(good_cols)
# train_leak, result = compiled_leak_result(extra_feats)
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

Processing lag 0
Leak values found in train 1388
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.51383333916
Processing lag 1
Leak values found in train 2010
% of correct leaks values in train  1.0
Score (filled with nonzero mean) 1.2798076585
Processing lag 2
Leak values found in train 2407
% of correct leaks values in train  0.999584545077
Score (filled with nonzero mean) 1.14727168219
Processing lag 3
Leak values found in train 2667
% of correct leaks values in train  0.999625046869
Score (filled with nonzero mean) 1.04762867686
Processing lag 4
Leak values found in train 2840
% of correct leaks values in train  0.999647887324
Score (filled with nonzero mean) 0.989106633692
Processing lag 5
Leak values found in train 2993
% of correct leaks values in train  0.99966588707
Score (filled with nonzero mean) 0.948694049968
Processing lag 6
Leak values found in train 3115
% of correct leaks values in train  0.999678972713
Score (filled with nonzero mean) 0.89949

In [12]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_correct,1.000000,1.000000,0.999585,0.999625,0.999648,0.999666,0.999679,0.999689,0.999697,0.999702,...,0.999477,0.999478,0.999478,0.999218,0.998959,0.998962,0.998704,0.998188,0.997672,0.997415
leaky_count,1388.000000,2010.000000,2407.000000,2667.000000,2840.000000,2993.000000,3115.000000,3217.000000,3303.000000,3357.000000,...,3822.000000,3828.000000,3834.000000,3838.000000,3844.000000,3853.000000,3859.000000,3864.000000,3866.000000,3869.000000
score,1.513833,1.279808,1.147272,1.047629,0.989107,0.948694,0.899494,0.851132,0.820565,0.800645,...,0.604919,0.602805,0.599573,0.597415,0.596461,0.594082,0.589793,0.588767,0.585366,0.585602


In [13]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
train_leak.head()

In [15]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [16]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000d6aaf2,38000000.0,38000000.0,38000000.0,0.0,38000000.0,0.0,38000000.0,0.0,0.0,...,38000000.0,0.0,38000000.0,0.0,38000000.0,38000000.0,0.0,0.0,38000000.0,38000000.0
1,000fbd867,600000.0,0.0,0.0,0.0,0.0,0.0,600000.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0
2,0027d6b71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0028cbf45,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2000000.0,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0
4,002a68644,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [18]:
def compiled_leak_result_test(max_nlags, extra_cols):
    test_leak = test
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, extra_cols, i)
        
        leaky_cols.append(c)
#         test_leak = test.join(
#             test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
#             on="ID", how="left"
#         )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [20]:
test_leak, test_result = compiled_leak_result_test(38, good_cols)

Processing lag 0
Leak values found in test 2965
Processing lag 1
Leak values found in test 4227
Processing lag 2
Leak values found in test 4967
Processing lag 3
Leak values found in test 5533
Processing lag 4
Leak values found in test 5961
Processing lag 5
Leak values found in test 6257
Processing lag 6
Leak values found in test 6467
Processing lag 7
Leak values found in test 6622
Processing lag 8
Leak values found in test 6787
Processing lag 9
Leak values found in test 6922
Processing lag 10
Leak values found in test 7037
Processing lag 11
Leak values found in test 7133
Processing lag 12
Leak values found in test 7205
Processing lag 13
Leak values found in test 7280
Processing lag 14
Leak values found in test 7339
Processing lag 15
Leak values found in test 7401
Processing lag 16
Leak values found in test 7450
Processing lag 17
Leak values found in test 7489
Processing lag 18
Leak values found in test 7520
Processing lag 19
Leak values found in test 7568
Processing lag 20
Leak values 

In [21]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
leaky_count,2965,4227,4967,5533,5961,6257,6467,6622,6787,6922,...,7765,7779,7795,7802,7810,7823,7842,7876,7935,8120


In [22]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [30]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

,ID,leaked_target_0,leaked_target_1,leaked_target_2,leaked_target_3,leaked_target_4,leaked_target_5,leaked_target_6,leaked_target_7,leaked_target_8,...,leaked_target_29,leaked_target_30,leaked_target_31,leaked_target_32,leaked_target_33,leaked_target_34,leaked_target_35,leaked_target_36,leaked_target_37,compiled_leak
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
# test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = 0

In [31]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

baseline_sub_lag_36.csv saved


In [25]:
sub.head()

,ID,target
0,000137c73,0.0
1,00021489f,0.0
2,0004d7953,0.0
3,00056a333,0.0
4,00056d8eb,0.0


In [32]:
# Mix with other pred
sub_no_leak = pd.read_csv('sub_138.csv')
ensemble_sub = sub.copy()
ensemble_sub[sub['target'] == 0] = sub_no_leak[sub['target'] == 0]
ensemble_sub.to_csv(f"submission1.csv", index=False)
print(f"ensemble saved")
ensemble_sub.head()

ensemble saved


,ID,target
0,000137c73,1.751810e+06
1,00021489f,1.229325e+06
2,0004d7953,2.285429e+06
3,00056a333,5.295290e+06
4,00056d8eb,1.816481e+06
